In [9]:

import configparser
import pandas as pd
import requests
import logging
import json
from rauth import OAuth1Service
from logging.handlers import RotatingFileHandler
import webbrowser
import datetime

In [10]:
#Read consumer key and consumer secret from config file
config_path = r'C:\Users\reedx\OneDrive\Investing\Python\EtradePythonClient\etrade_python_client\config.ini'
config = configparser.ConfigParser()
config.read(config_path)

consumer_key = config['DEFAULT']['CONSUMER_KEY']
consumer_secret = config['DEFAULT']['CONSUMER_SECRET']


In [11]:
def default_oauth():
    etrade = OAuth1Service(
        name="etrade",
        consumer_key = consumer_key,
        consumer_secret = consumer_secret,
        request_token_url="https://api.etrade.com/oauth/request_token",
        access_token_url="https://api.etrade.com/oauth/access_token",
        authorize_url="https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
        base_url="https://api.etrade.com")

    menu_items = {"1": "Live Consumer Key",
                  "2": "Exit"}
    while True:
        print("")
        options = menu_items.keys()
        for entry in options:
            print(entry + ")\t" + menu_items[entry])
        selection = input("Please select Consumer Key Type: ")
        if selection == "1":
            base_url = config["DEFAULT"]["PROD_BASE_URL"]          
            break
        elif selection == "2":
            break
        else:
            print("Unknown Option Selected!")
    print("")

    # Step 1: Get OAuth 1 to request token and secret
    request_token, request_token_secret = etrade.get_request_token(
        params={"oauth_callback": "oob", "format": "json"})

    # Step 2: Go through the authentication flow. Login to E*TRADE.
    # After you login, the page will provide a verification code to enter.
    authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
    webbrowser.open(authorize_url)
    text_code = input("Please accept agreement and enter verification code from browser: ")

    # Step 3: Start an authenticated section by entering verification code
    session = etrade.get_auth_session(request_token,
                                  request_token_secret,
                                  params={"oauth_verifier": text_code})

    print("Successful authorization")
    return session, base_url


In [12]:
#Start authorized session
session, base_url = default_oauth()


1)	Live Consumer Key
2)	Exit


Please select Consumer Key Type:  1


Please accept agreement and enter verification code from browser:  2XVJE


Successful authorization


In [13]:
#Set up logger settings
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)
handler = RotatingFileHandler("python_client.log", maxBytes=5 * 1024 * 1024, backupCount=3)
FORMAT = "%(asctime)-15s %(message)s"
fmt = logging.Formatter(FORMAT, datefmt='%m/%d/%Y %I:%M:%S %p')
handler.setFormatter(fmt)
logger.addHandler(handler)

In [21]:
class My_Accounts:

    def __init__(self, session, base_url):
        #Initialize Accounts object with session and account information
        #param session: authenticated session

        self.session = session
        self.account = {}
        self.base_url = base_url

    def get_account_details(self,account_index):
        
        #Calls account list API to retrieve a list of the user's E*TRADE accounts
        #param self:Passes in parameter authenticated session

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/list.json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            
            if data is not None and "AccountListResponse" in data and "Accounts" in data["AccountListResponse"] \
                    and "Account" in data["AccountListResponse"]["Accounts"]:
                accounts = data["AccountListResponse"]["Accounts"]["Account"]

                accounts[:] = [d for d in accounts if d.get('accountStatus') != 'CLOSED']
                account_ID = accounts[account_index]['accountId']
                account_ID_key = accounts[account_index]['accountIdKey']
                institution_type = accounts[account_index]['institutionType']

            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + data["Error"]["message"])
                else:
                    print("Error: AccountList API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: AccountList API service error")
        return account_ID, account_ID_key, institution_type

    def balance(self, account_ID_key,institution_type):

        #Calls account balance API to retrieve the current balance and related details for a specified account
        #param self: Pass in parameters authenticated session and information on selected account

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/balance.json"

        # Add parameters and header information
        params = {"instType": institution_type, "realTimeNAV": "true"}
        headers = {"consumerkey": consumer_key}

        # Make API call for GET request
        response = self.session.get(url, header_auth=True, params=params, headers=headers)
        logger.debug("Request url: %s", url)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            if data is not None and "BalanceResponse" in data:
                balance_data = data["BalanceResponse"]
                if balance_data is not None and "accountId" in balance_data:
                    print("\n\nBalance for " + balance_data["accountId"] + ":")
                else:
                    print("\n\nBalance:")
                # Display balance information
                if balance_data is not None and "accountDescription" in balance_data:
                    print("Account Nickname: " + balance_data["accountDescription"])
                if balance_data is not None and "Computed" in balance_data \
                        and "RealTimeValues" in balance_data["Computed"] \
                        and "totalAccountValue" in balance_data["Computed"]["RealTimeValues"]:
                    print("Net Account Value: "
                          + str('${:,.2f}'.format(balance_data["Computed"]["RealTimeValues"]["totalAccountValue"])))
                if balance_data is not None and "Computed" in balance_data \
                        and "marginBuyingPower" in balance_data["Computed"]:
                    print("Margin Buying Power: " + str('${:,.2f}'.format(balance_data["Computed"]["marginBuyingPower"])))
                if balance_data is not None and "Computed" in balance_data \
                        and "cashBuyingPower" in balance_data["Computed"]:
                    print("Cash Buying Power: " + str('${:,.2f}'.format(balance_data["Computed"]["cashBuyingPower"])))
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")

    def portfolio(self,account_ID_key):
        
        #Call portfolio API to retrieve a list of positions held in the specified account (pulls 50 transactions at a time, 2nd loop handles 51+)
        #param self: Passes in parameter authenticated session and information on selected account
        

        # URL for the API endpoint
        url =self.base_url + "/v1/accounts/" + account_ID_key + "/portfolio.json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        logger.debug("Request Header: %s", response.request.headers)

        print("\nPortfolio:")

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()

            if data is not None and "PortfolioResponse" in data and "AccountPortfolio" in data["PortfolioResponse"]:
                # Display balance information
                for acctPortfolio in data["PortfolioResponse"]["AccountPortfolio"]:
                    if acctPortfolio is not None and "Position" in acctPortfolio:
                        for position in acctPortfolio["Position"]:
                            print_str = ""
                            if position is not None and "symbolDescription" in position:
                                print_str = print_str + "Symbol: " + str(position["symbolDescription"])
                            if position is not None and "quantity" in position:
                                print_str = print_str + " | " + "Quantity #: " + str(position["quantity"])
                            if position is not None and "Quick" in position and "lastTrade" in position["Quick"]:
                                print_str = print_str + " | " + "Last Price: " \
                                            + str('${:,.2f}'.format(position["Quick"]["lastTrade"]))
                            if position is not None and "pricePaid" in position:
                                print_str = print_str + " | " + "Price Paid $: " \
                                            + str('${:,.2f}'.format(position["pricePaid"]))
                            if position is not None and "totalGain" in position:
                                print_str = print_str + " | " + "Total Gain $: " \
                                            + str('${:,.2f}'.format(position["totalGain"]))
                            if position is not None and "marketValue" in position:
                                print_str = print_str + " | " + "Value $: " \
                                            + str('${:,.2f}'.format(position["marketValue"]))
                            print(print_str)
                    else:
                        print("None")
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and "headers" in response and "Content-Type" in response.headers \
                        and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Portfolio API service error")
        elif response is not None and response.status_code == 204:
            print("None")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and "headers" in response and "Content-Type" in response.headers \
                    and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Portfolio API service error")
                
    def get_past_transactions(self,account_ID_key,start,end,buy_sell):
        
        #Call transactions API to retrieve a list of transactions
        #Organize purchases over last 30 days in a dataframe
        #param self: Passes in parameter authenticated session and information on selected account
                
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/transactions.json"

        headers = {"consumerkey": consumer_key}
        parameters = {"startDate":start,"endDate":end,"count":50}

        response_executed = session.get(url, header_auth=True, params = parameters, headers=headers)

        logger.debug("Request Header: %s", response_executed.request.headers)
        logger.debug("Response Body: %s", response_executed.text)
        logger.debug(response_executed.text)

        # Handle and parse response
        if response_executed.status_code == 204:
            logger.debug(response_executed)
            print("None")
        elif response_executed.status_code == 200:
            parsed = json.loads(response_executed.text)
            logger.debug(json.dumps(parsed, indent=4, sort_keys=True))

            purchases_30d = pd.DataFrame(columns=['Ticker','Cost','Date'])

            data = response_executed.json()

            transactions = data["TransactionListResponse"]["Transaction"]
            transaction_count = data['TransactionListResponse']['transactionCount']
            
            try:
                marker_value = data['TransactionListResponse']['marker'] #marker used to pull next set of transactions
            except:
                marker_value = 'No marker, this is the last pull'
            
            global marker
            marker = marker_value         
            
            transactions[:] = [d for d in transactions if d.get('transactionType') == buy_sell]

            for i in range(len(transactions)):
                ticker = transactions[i-1]['brokerage']['product']['symbol']
                cost = transactions[i-1]['brokerage']['price']
                date = transactions[i-1]['transactionDate']
                date = datetime.datetime.fromtimestamp(date/1000)
                new_row = pd.DataFrame({'Ticker':[ticker], 'Cost':[cost],'Date':[date]})

                purchases_30d = pd.concat([purchases_30d,new_row], ignore_index = True)
                
            #Another loop to pull transactions again
            #Pulls transactions 51+ (can only pull 50 transactions at a time)
            #Changes to pull those transactions:
            #Adding 'marker':marker in the api pull
            #Take existing purchases_30d from prior loop and adding rows
            while transaction_count == 50:
                url = self.base_url + "/v1/accounts/" + account_ID_key + "/transactions.json"

                headers = {"consumerkey": consumer_key}
                parameters = {"startDate":start,"endDate":end,"count":50,'marker':marker}

                response_executed = session.get(url, header_auth=True, params = parameters, headers=headers)

                print(response_executed)

                logger.debug("Request Header: %s", response_executed.request.headers)
                logger.debug("Response Body: %s", response_executed.text)
                logger.debug(response_executed.text)

                print("\nTransactions last 30d:")
                # Handle and parse response
                if response_executed.status_code == 204:
                    logger.debug(response_executed)
                    print("None")
                elif response_executed.status_code == 200:
                    parsed = json.loads(response_executed.text)
                    logger.debug(json.dumps(parsed, indent=4, sort_keys=True))
                    data = response_executed.json()

                    transactions = data["TransactionListResponse"]["Transaction"]
                    
                    try:
                        marker_value = data['TransactionListResponse']['marker'] #marker used to pull next set of transactions
                    except:
                        marker_value = 'No marker, this is the last pull'
                        
                    transaction_count = data['TransactionListResponse']['transactionCount']
                    marker = marker_value

                    transactions[:] = [d for d in transactions if d.get('transactionType') == buy_sell]

                    for i in range(len(transactions)):
                        ticker = transactions[i-1]['brokerage']['product']['symbol']
                        cost = transactions[i-1]['brokerage']['price']
                        date = transactions[i-1]['transactionDate']
                        date = datetime.datetime.fromtimestamp(date/1000)
                        new_row = pd.DataFrame({'Ticker':[ticker], 'Cost':[cost],'Date':[date]})
                        purchases_30d = pd.concat([purchases_30d,new_row], ignore_index = True)

            return purchases_30d                     

    def portfolio(self,account_ID_key):
        
        #Call portfolio API to retrieve a list of positions held in the specified account
        #param self: Passes in parameter authenticated session and information on selected account

        # URL for the API endpoint
        url =self.base_url + "/v1/accounts/" + account_ID_key + "/portfolio.json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        logger.debug("Request Header: %s", response.request.headers)

        #Handle response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()

            if data is not None and "PortfolioResponse" in data and "AccountPortfolio" in data["PortfolioResponse"]:

                current_portfolio_df = pd.DataFrame(columns=['Ticker','Quantity','Last Price','Total Value'])

                # Display balance information
                for acctPortfolio in data["PortfolioResponse"]["AccountPortfolio"]:
                    if acctPortfolio is not None and "Position" in acctPortfolio:
                        for position in acctPortfolio["Position"]:
                            if position is not None and "symbolDescription" in position:
                                ticker = str(position['symbolDescription'])
                            if position is not None and "quantity" in position:
                                quantity = position['quantity']
                            if position is not None and "Quick" in position and "lastTrade" in position["Quick"]:
                                last_price = position['Quick']['lastTrade']
                            if position is not None and "marketValue" in position:
                                total_value = position['marketValue']
                            #can also add 'pricePaid' and 'totalGain' from position data

                            new_row = pd.DataFrame({'Ticker':[ticker], 'Quantity':[quantity],'Last Price':[last_price], 'Total Value':[total_value]})
                            current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)
                            
                    else:
                        print("None")
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and "headers" in response and "Content-Type" in response.headers \
                        and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Portfolio API service error")
        elif response is not None and response.status_code == 204:
            print("None")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and "headers" in response and "Content-Type" in response.headers \
                    and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Portfolio API service error")
        return current_portfolio_df
    
    def get_balance(self, account_ID_key,institution_type):
        
        #Calls account balance API to retrieve the current balance and related details for a specified account
        #param self: Pass in parameters authenticated session and information on selected account

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/balance.json"

        # Add parameters and header information
        params = {"instType": institution_type, "realTimeNAV": "true"}
        headers = {"consumerkey": consumer_key}

        # Make API call for GET request
        response = self.session.get(url, header_auth=True, params=params, headers=headers)
        logger.debug("Request url: %s", url)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            if data is not None and "BalanceResponse" in data:
                balance_data = data["BalanceResponse"]
                # Display balance information
                if balance_data is not None and "Computed" in balance_data \
                        and "RealTimeValues" in balance_data["Computed"] \
                        and "totalAccountValue" in balance_data["Computed"]["RealTimeValues"]:
                    print("Net Account Value: " + str('${:,.2f}'.format(balance_data["Computed"]["RealTimeValues"]["totalAccountValue"])))
                    account_balance = balance_data["Computed"]["RealTimeValues"]["totalAccountValue"]

                if balance_data is not None and "Computed" in balance_data \
                        and "cashBuyingPower" in balance_data["Computed"]:
                    print("Cash Buying Power: " + str('${:,.2f}'.format(balance_data["Computed"]["cashBuyingPower"])))
                    account_cash_buying_power = balance_data["Computed"]["cashBuyingPower"]
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")
        return account_balance, account_cash_buying_power

    def get_quote(self,symbols):
        
        #Calls account balance API to retrieve the current balance and related details for a specified account
        #param self: Pass in parameters authenticated session and information on selected account
        
        quote_df = pd.DataFrame(columns = ['Ticker','ask','bid','last_trade','last_trade_time','last_trade_minutes_ago'])
        
        # URL for the API endpoint
        url = self.base_url + "/v1/market/quote/" + symbols + ".json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        
        logger.debug("Request url: %s", url)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            
            if data is not None and "QuoteResponse" in data:
                quote_response = data["QuoteResponse"]
                # Display balance information
                if quote_response is not None and "QuoteData" in quote_response:
                    if quote_response is not None and "QuoteData" in quote_response:
                        
                        quote_data = quote_response["QuoteData"] #this is where you can iterate for multiple quotes
                        for quote in quote_data:
                            ticker = quote['Product']['symbol']
                            ask = quote['All']['ask']
                            bid = quote['All']['bid']
                            last_trade = quote['All']['lastTrade']
                            last_trade_time = quote['All']['timeOfLastTrade']
                            last_trade_time = datetime.datetime.fromtimestamp(last_trade_time)
                            last_trade_minutes_ago = (datetime.datetime.now() - last_trade_time).total_seconds()/60
                            
                            new_row = {'Ticker':ticker,'ask':ask,'bid':bid,
                                       'last_trade':last_trade,'last_trade_time':last_trade_time,'last_trade_minutes_ago':last_trade_minutes_ago}
                            new_row = pd.DataFrame([new_row])
                            quote_df = pd.concat([quote_df,new_row],ignore_index=True)
                            
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")
        return quote_df

In [25]:
#1. Get account ID
#2. Use account ID to get account balance
#3. Get existing positions


accounts = My_Accounts(session, base_url)

#Get account ID (used for all other account details)
account_ID, account_ID_key, institution_type = accounts.get_account_details(0) #Entering 0 here to get the ID for the 1st account

#Get account balance and cash balance
account_balance, account_cash_buying_power = accounts.get_balance(account_ID_key, institution_type)

#Get current portfolio              
current_portfolio_df = accounts.portfolio(account_ID_key)

current_portfolio_df


Net Account Value: $10,718.36
Cash Buying Power: $149.85


C:\Users\reedx\AppData\Local\Temp\ipykernel_19396\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)


,Ticker,Quantity,Last Price,Total Value
0,DJP,17,32.420,551.1399
1,TIP,16,106.600,1705.5999
2,XLE,5,88.030,440.1499
3,XLI,3,132.820,398.4599
4,XLP,9,77.380,696.4199
5,XLU,7,76.010,532.0700
6,XLV,6,140.430,842.5800
7,SPLV,11,69.560,765.1599
8,SLV,7,27.400,191.8000
9,EEM,9,41.800,376.1999


In [19]:
#GET LAST 30 DAYS OF TRANSACTIONS INTO DATAFRAME

today = datetime.date.today()
thirty_days_ago = today - datetime.timedelta(days=31)

today = today.strftime('%m%d%Y')
thirty_days_ago = thirty_days_ago.strftime('%m%d%Y')

accounts = My_Accounts(session, base_url)            
purchases_30d = accounts.get_past_transactions(account_ID_key,thirty_days_ago,today,'Bought')
sales_30d = accounts.get_past_transactions(account_ID_key,thirty_days_ago,today,'Sold')

print("purchases_30d")
print(purchases_30d)
print("sales_30d")
print(sales_30d)

C:\Users\reedx\AppData\Local\Temp\ipykernel_19396\1918338655.py:237: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  purchases_30d = pd.concat([purchases_30d,new_row], ignore_index = True)


purchases_30d
  Ticker      Cost                Date
0    XLV  137.8050 2024-12-31 12:04:39
1    DJP   32.2632 2025-01-06 17:19:56
2    XLU   76.4300 2025-01-06 17:18:46
3    XLE   87.1300 2025-01-06 17:18:45
4    TIP  106.4980 2025-01-06 17:18:32
5   SPLV   70.1150 2024-12-31 12:14:40
6    XLP   78.6200 2024-12-31 12:14:00
sales_30d
   Ticker      Cost                Date
0     XLV  137.5800 2024-12-31 11:30:00
1     EEM   42.3379 2025-01-06 17:17:48
2     GLD  243.1500 2025-01-06 17:16:14
3     EFA   76.3740 2025-01-06 17:16:10
4     SLV   27.2844 2025-01-06 17:16:09
5    GOVT   22.4305 2025-01-06 17:17:49
6    SPLV   69.2350 2025-01-06 17:16:13
7     XLP   77.6020 2025-01-06 17:16:12
8     XLY  225.8200 2025-01-06 17:16:08
9     XLI  132.3721 2025-01-06 17:16:08
10    IWM  224.5301 2025-01-06 17:16:10
11   SPLV   70.0500 2024-12-31 11:30:00
12    XLP   78.5300 2024-12-31 11:30:00


C:\Users\reedx\AppData\Local\Temp\ipykernel_19396\1918338655.py:237: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  purchases_30d = pd.concat([purchases_30d,new_row], ignore_index = True)
